In [ ]:
from lib.utils import *
import plotly.express as px
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,TensorDataset
from torch import nn
from torch.nn.functional import relu
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.metrics import f1_score,recall_score,precision_score
import seaborn as sns

In [ ]:
skip_idx = [19,24,26,32,34,38,40,45,52,55,70]
train_idx = [0]
X,y = load_nursing_by_index(0)
X = X[:,0].unsqueeze(1)
torch.manual_seed(1337)
batch_size = 4
block_size = 5
def get_batch():
    ix = torch.randint(len(X)-block_size,(batch_size,))
    xi = torch.stack([X[i:i+block_size] for i in ix])
    yi = torch.stack([y[i:i+block_size] for i in ix])
    return xi,yi
xb,yb = get_batch()
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(context,target)

In [ ]:
xb.shape

In [ ]:
class MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=3,hidden_size=4,bidirectional=True)
        self.fc1 = nn.Linear(8,1)
    
    def forward(self, x):
        o,_ = self.lstm(x)
        x = self.fc1(o)
        return x,o
device = 'cuda'
model = MODEL().to(device=device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

In [ ]:
lossi = []
model.train()

for i in tqdm(range(100)):
    logits,_ = model(X.cuda())
    loss = criterion(logits,y.cuda())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    lossi.append(loss.item())
plt.plot(lossi)
print(torch.tensor(lossi).mean())

In [ ]:
lossi = []
f1i = []
recalli = []
precisioni = []
for idx in tqdm(range(71)):
    if(idx in skip_idx or idx in train_idx):
        continue
    Xi,yi = load_nursing_by_index(idx)
    logits = model(Xi.cuda())[0]
    lossi.append(criterion(logits,yi.cuda()).item())
    y_pred = torch.sigmoid(logits).detach().cpu().round().numpy()
    f1i.append(f1_score(y_true=yi,y_pred=y_pred,average='macro'))
    recalli.append(recall_score(y_true=yi,y_pred=y_pred,average='macro'))
    precisioni.append(precision_score(y_true=yi,y_pred=y_pred,average='macro'))

In [ ]:
sns.kdeplot(f1i,bw_adjust=.4,label='f1')
print('f1',torch.tensor(f1i).mean()) #.5579
print('f1',torch.tensor(f1i).std()) #.1294
sns.kdeplot(recalli,bw_adjust=.4,label='recall')
print('recall',torch.tensor(recalli).mean()) #.7122
print('recall',torch.tensor(recalli).std()) #.1294
sns.kdeplot(precisioni,bw_adjust=.4,label='precision')
print('precision',torch.tensor(precisioni).mean()) #.7122
print('precision',torch.tensor(precisioni).std()) #.1294
plt.legend()

In [ ]:
feature_maps = model.c1(X.T.cuda())
fig,axes = plt.subplots(nrows=len(feature_maps),ncols=1,figsize=(20,20))
for i in range(len(feature_maps)):
    axes[i].plot(feature_maps[i][25000:].detach().cpu())
plt.savefig('kernels.jpg',dpi=500)

In [ ]:
Xt,yt = load_nursing_by_index(5)
df = pd.DataFrame(Xt.numpy())
df['y'] = yt.numpy()*10
logits,o = model(Xt.cuda())
df['y_pred'] = torch.sigmoid(logits).detach().cpu().round().numpy()*10
# df[['h1','h2','h3','h4','h5','h6','h7','h8']] = o.detach().cpu().numpy()*5+30
fig = px.line(df)
fig.show(renderer='browser')

In [ ]:
lossi = []
model.train()

for i in range(1):
    for (X_tr,y_tr) in tqdm(trainloader):
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi.append(loss.item())
plt.plot(lossi)
print(torch.tensor(lossi).mean())

In [ ]:
loss_tr = []
loss_dev = []
pbar = tqdm(range(5))

for epoch in pbar:
    # train loop
    model.train()
    loss_tr_total = 0
    for (X_tr,y_tr) in trainloader:
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tr_total += loss.item()
    loss_tr.append(loss_tr_total/len(trainloader))

    # dev loop
    model.eval()
    loss_dev_total = 0
    for (X_dv,y_dv) in devloader:
        X_dv,y_dv = X_dv.to(device),y_dv.to(device)
        logits = model(X_dv)
        loss = criterion(logits,y_dv)
        loss_dev_total += loss.item()
    loss_dev.append(loss_dev_total/len(devloader))
plt.plot(loss_tr,label='train')
plt.plot(loss_dev,label='dev')
plt.legend()

In [ ]:
_,_,y_pred = test_evaluation(trainloader,model,criterion)
_,_,y_pred = test_evaluation(devloader,model,criterion)
# train : .0749 
# dev : .0785

In [ ]:
idx = 0

holdoutloader = DataLoader(TensorDataset(*load_and_window_nursing_by_index(idx,window_size=window_size)),batch_size=32,shuffle=False)
loss,y_true,y_pred = test_evaluation(holdoutloader,model,criterion)
X,y = load_nursing_by_index(idx)
df = pd.DataFrame(X.numpy())
df['y_pred'] = torch.cat([torch.zeros(window_size//2),y_pred.flatten().round(),torch.zeros(window_size//2)])*10
df['y_true'] = y*10
fig = px.line(df)
fig.show(renderer='browser')

In [ ]:
f1i = []
for idx in tqdm(range(71)):
    if(idx in skip_idx or idx in train_idx):
        continue
    X,y = load_and_window_nursing_by_index(idx,window_size=window_size)
    y_pred = torch.sigmoid(model(X.cuda())).round().detach().cpu()
    f1i.append(f1_score(y_true=y,y_pred=y_pred,average='macro'))

sns.kdeplot(f1i,bw_adjust=.4)
print(torch.tensor(f1i).mean()) #.7122
print(torch.tensor(f1i).std()) #.1294
